In [ ]:
! pip uninstall tensorflow
! pip install tensorflow-gpu==2.0.0

Uninstalling tensorflow-2.5.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.5.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.5.0
     |████████████████████████████████| 380.8MB 45kB/s 
     |████████████████████████████████| 450kB 38.3MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 3.8MB 43.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7557 sha256=e02bd971722f390533d4ec4edb24a1928b3d4c3cc3c97eefca4bdc1e3168cbaf
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successf

In [ ]:
# import tensorflow libraries
# check a tensorflow version

import tensorflow as tf
tf.__version__

'2.0.0'

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# load an MNIST dataset
# MNIST image: 28 x 28 (gray scale)
img_rows, img_cols = 28, 28
(x_train, y_train), (x_test, y_test) = mnist.load_data()


11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
# CNN: input dimension = 3
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [ ]:
print(x_train.dtype)
print(x_test.dtype)

uint8
uint8


In [ ]:
# convert data type to float32 for train
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# normalization
x_train = x_train / 255.0
x_test = x_test / 255.0

In [ ]:
# y_train, y_test : integer value between 0 and 9
print(y_train.shape)

(60000,)


In [ ]:
# y_train, y_test : integer value --> vector value
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

print(y_train.shape)

(60000, 10)


In [ ]:
# make a simple CNN model
model = Sequential()
model.add(Conv2D(
    filters=32,
    kernel_size=(3, 3),
    activation='relu',
    input_shape=input_shape
))
model.add(Conv2D(
    filters=64,
    kernel_size=(3,3),
    activation='relu'
))
model.add(
    MaxPool2D(
        pool_size=(2, 2)
    )
)
model.add(
    Dropout(0.25)
)
model.add(
    Flatten()
)
model.add(
    Dense(
        units=128,
        activation='relu'
    )
)
model.add(
    Dropout(0.5)
)
model.add(
    Dense(
        units=num_classes,
        activation='softmax'
    )
)
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
# make a data generator
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest'
)

In [ ]:
# train
batch_size = 128
epochs = 1
model.fit(
    datagen.flow(x_train, y_train, batch_size=batch_size),
    validation_data=(x_test, y_test),
    epochs=epochs,
    verbose=1
)

Train for 469 steps, validate on 10000 samples
469/469 [==============================] - 23s 49ms/step - loss: 1.0990 - accuracy: 0.6307 - val_loss: 0.1681 - val_accuracy: 0.9544


In [ ]:
score = model.evaluate(
    x_test, y_test, verbose=1
)

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [ ]:
# save model to the local disk
model.save('mnist_model.h5')

from google.colab import files
files.download('mnist_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# convert the model to tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('mnist_model.tflite', 'wb') as f:
    f.write(tflite_model)

files.download('mnist_model.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>